In [1]:
from Carcassonne_Game.Carcassonne import CarcassonneState
from TicTacToe_Game.TicTacToe import TicTacToeState 
from Function_Optimisation_Game.Function_Optimisation import FunctionOptimisationState
from player.Player import RandomPlayer
from player.MCTS_Player import MCTSPlayer
from player.MCTS_RAVE_Player import MCTS_RAVEPlayer
from player.MCTS_ES_BACK_Player import MCTS_ES_BACK_Player
from player.MCTS_ES_BACK_SEM_Player import MCTS_ES_BACK_SEM_Player

import Experimental_Setup as exps
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
#from plotly.tools import make_subplots

import os
import pandas as pd
from datetime import date
from collections import OrderedDict
import time
import multiprocessing as mp
import numpy as np
import ast
import random
import math
import statistics as stats
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats import bernoulli
from scipy import optimize
from itertools import repeat

In [ ]:
def fo_experiment(func_index,
                  random_seed,
                  experiment_type="1p",
                  player_dicts = [{"type":"VMCTS"}],
                  runs = 30,
                  splits = 2,
                  ranges = [[0,1]],
                  minimum_step=0.00001,
                  #Logs data
                  log_path = ["FO","Default_folder"],
                  logs_divisions=3,
                  tree_data=True,
                  ):
   """
   experiment_type can be 1p or 2p
   player_dicts is a list of dictionaries. Each dictionary must contain the key "type" with a value. The value can only be: "VMCTS","SIEAMCTS or "RANDOM"
   """

   #Setup
   logfile=os.path.join(*log_path)
   if not os.path.exists(logfile):
      os.makedirs(logfile)
   random.seed(random_seed)
   np.random.seed(seed=random_seed)

   #Players
   players={}
   for player_dict_index in range(len(player_dicts)):

      if player_dicts[player_dict_index]["type"] == "VMCTS":
         default_player_dict = { "iterations":5000,
                                    "c":math.sqrt(2),
                                    "name":"Vanilla_MCTS",
                                    "rollouts":1,
                                    "logs":True,
                                  }
         default_player_dict.update(player_dicts[player_dict_index])
         player_dicts[player_dict_index] = default_player_dict

         players[player_dict_index] = MCTSPlayer(iterations=default_player_dict["iterations"]
                                 ,c_param = default_player_dict["c"]
                                 ,name= default_player_dict["name"]
                                 ,rollouts = default_player_dict["rollouts"]
                                 ,logs=default_player_dict["logs"]
                                 ,logfile=logfile)


      elif player_dicts[player_dict_index]["type"] == "SIEAMCTS":
         default_player_dict = {"iterations":5000,
                                    "name":"SIEA_MCTS",
                                    "rollouts":1,
                                    "lambda" : 4,
                                    "generations" : 20,
                                    "es_sims" : 30,
                                    "seml" : 0.1,
                                    "semu" : 0.5,
                                    "estype" : "comma",
                                    "c":math.sqrt(2),
                                    "logs":True,
                                  }
         default_player_dict.update(player_dicts[player_dict_index])
         player_dicts[player_dict_index] = default_player_dict

         players[player_dict_index] = MCTS_ES_BACK_SEM_Player(iterations=default_player_dict["iterations"]
                              ,c_param=default_player_dict["c"]
                              ,rollouts = default_player_dict["rollouts"]
                              ,name=default_player_dict["name"]
                              ,Lambda=default_player_dict["lambda"]
                              ,NGen=default_player_dict["generations"]
                              ,ES_Sims=default_player_dict["es_sims"]
                              ,ESType=default_player_dict["es_type"]
                              ,logs=default_player_dict["logs"]
                              ,logfile=logfile
                              ,Sem_L=default_player_dict["seml"]
                              ,Sem_U=default_player_dict["semu"])
         
      elif player_dicts[player_dict_index]["type"] == "EAMCTS":
         default_player_dict = {"iterations":5000,
                                    "name":"EA_MCTS",
                                    "rollouts":1,
                                    "lambda" : 4,
                                    "generations" : 20,
                                    "es_sims" : 30,
                                    "es_type" : "comma",
                                    "c":math.sqrt(2),
                                    "logs":True,
                                  }
         default_player_dict.update(player_dicts[player_dict_index])
         player_dicts[player_dict_index] = default_player_dict

         players[player_dict_index] = MCTS_ES_BACK_SEM_Player(iterations=default_player_dict["iterations"]
                              ,c_param=default_player_dict["c"]
                              ,rollouts = default_player_dict["rollouts"]
                              ,name=default_player_dict["name"]
                              ,Lambda=default_player_dict["lambda"]
                              ,NGen=default_player_dict["generations"]
                              ,ES_Sims=default_player_dict["es_sims"]
                              ,ESType=default_player_dict["es_type"]
                              ,logs=default_player_dict["logs"]
                              ,logfile=logfile)

      elif player_dicts[player_dict_index]["type"] == "RANDOM":
         players[player_dict_index] = RandomPlayer()

      else: print("Wrong player type")

   #Problem
   state = FunctionOptimisationState(players=players, function=func_index, ranges=ranges, splits=splits, minimum_step=minimum_step)# give any single player
   used_func = state.function

   #Execution
   if experiment_type == "1p":
      all_data = []
      all_fo_logs = pd.DataFrame()
      for p_id, player in players.items():
         fo_logs, data = exps.multiple_runs(state, used_func, player, runs, random_seed, logs_divisions)
         all_fo_logs = pd.concat([all_fo_logs,fo_logs])
         all_data.append(data)
         all_data_df = pd.concat(all_data)
         all_data_df.to_csv(os.path.join('logs', logfile, "Tree_data.csv"), index=False)

      #Create final logs
      #Combine separate logs
      exps.CombineFiles(logs=True,logfile=logfile)

      #Results logs
      player_logs = defaultdict(lambda:[])
      for player_idx, player in players.items():
         temp_logs = all_fo_logs[all_fo_logs["Player"]==player.name]
         player_logs["Player"].append(player.name)
         player_logs["Mean_Tree_Nodes"].append(temp_logs["Tree_Nodes"].mean())
         player_logs["Std_Tree_Nodes"].append(temp_logs["Tree_Nodes"].std())
         player_logs["Mean_Terminals_Reached"].append(temp_logs["Terminals_Reached"].mean())
         player_logs["Std_Terminals_Reached"].append(temp_logs["Terminals_Reached"].std())
         player_logs["Mean_Max_Visits_Path"].append(temp_logs["Max_Visits_Path"].mean())
         player_logs["Std_Max_Visits_Path"].append(temp_logs["Max_Visits_Path"].std())
         player_logs["Mean_Max_Reward_Path"].append(temp_logs["Max_Reward_Path"].mean())
         player_logs["Std_Max_Reward_Path"].append(temp_logs["Max_Reward_Path"].std())
         player_logs["Tree_Reaches_Terminal"].append(temp_logs["Max_Reward_Path"].values.sum()/len(temp_logs))

      player_df = pd.DataFrame(player_logs)
      player_df.to_csv(os.path.join('logs', logfile, "Final_Player_logs" + '.csv'), index=False)
      all_fo_logs.to_csv(os.path.join('logs', logfile, "Player_logs" + '.csv'), index=False)
   
   #Parameters logs
   final_logs={}
   final_logs["experiment_type"]=experiment_type
   final_logs["runs"]=runs
   final_logs["random_seed"]=random_seed
   final_logs["logfile"]=logfile
   #Problem parameters
   final_logs["branching_factor"]=splits
   final_logs["func_index"]=func_index
   final_logs["ranges"]=ranges
   #Player parameters
   for p_index,playeri in players.items():
      for k,v in player_dicts[p_index].items():
         final_logs["Player_" + str(p_index) + "_" + k] = v

   final_df = pd.DataFrame(final_logs)
   final_df.to_csv(os.path.join('logs', logfile, "Parameter_logs" + '.csv'), index=False)

#for f in [5,6,7,8,9]:
for f in [0,1,2,3,4]:
   for its in [2600, 5000]:

   #for c in [0.5,1,math.sqrt(2),2,3]:
      fo_experiment(func_index=f,
                     random_seed=0,
                     experiment_type="1p",
                     player_dicts = [{"type":"EAMCTS", "iterations":its, "c":math.sqrt(2)}],
                     runs = 30,
                     splits = 2,
                     ranges = [[0,1]],
                     minimum_step=0.00001,
                     #Logs data
                     log_path = ["FO","EAMCTS_f"+str(f)+"_its_"+str(its)],
                     logs_divisions=3,
                     tree_data=True,)


In [ ]:
def Collect_FO_logs(logs_path = "logs/FO", output_name = "collective_logs.csv", exp_names=None, include_tree_logs = False,  output_tree_name = "collective_logs.csv"):
      """
      1 player
      Collects data in "collective_tree_logs.csv".
      Logs names should be saved as "logs_path/Results_f0_c0.5" where f is the function, c is the parameter
      Logs in that folder should contain "Final_Player_logs.csv" and "Parameter_logs.csv"
      """
         
      if exp_names is None:
         exp_names = [ item for item in os.listdir(logs_path) if os.path.isdir(os.path.join(logs_path, item)) ]
          
      final_df = pd.DataFrame()
      final_tree_df = pd.DataFrame()
      for i,exp_name in enumerate(exp_names):
         data = pd.read_csv(os.path.join(logs_path, exp_name, "Final_Player_logs.csv"))
         pars = pd.read_csv(os.path.join(logs_path, exp_name, "Parameter_logs.csv"))
         tree_data = pd.read_csv(os.path.join(logs_path, exp_name, "Tree_data.csv"))

         if "Player_0_name" in pars.columns:
             player_name = pars["Player_0_name"][0]
         else:
             player_name = exp_name.split("_f")[0]
         if "func_index" in pars.columns:
             func_index = pars["func_index"][0]
         else:
             func_index = exp_name.split("_f")[1].split("_")[0]
         if "Player_0_c" in pars.columns or "c_param" in pars.columns:
             if "Player_0_c" in pars.columns: c_param = pars["Player_0_c"]

         for par_col in pars.columns:


         fi_list = [pars["func_index"][0] for _ in range(len(data))]
         c_list = [pars["c_param"][0] for _ in range(len(data))]
         expname_list = [exp_name for _ in range(len(data))]
         data["function"] = fi_list
         data["c"] = c_list
         data["expname"] = expname_list

         exp_names = [item for item in os.listdir(logs_path) if os.path.isdir(os.path.join(logs_path, item))]
         for i,exp_name in enumerate(exp_names):
            f_index = int(exp_name.split("_f")[-1][0])
            all_f.append(f_index)
            c = float(exp_name.split("_c")[-1])
            all_c.append(c)
            data = pd.read_csv(logs_path + join + exp_name + join + "Tree_data.csv")
            data["c_param"] = [c for _ in range(len(data))]
            data["f_index"] = [f_index for _ in range(len(data))]
            all_data.append(data)

         all_f = set(all_f)
         all_c = set(all_c)
         final_df = pd.concat(all_data)
         final_df.to_csv(logs_path + join + output_name, index=False)

         if i==0:
            final_df = pd.DataFrame(data)
         else:
            final_df = pd.concat([final_df,data]) 
      
      final_df.to_csv(logs_path + join + output_name, index=False)

In [ ]:
#plots to compare different algorithms in the same function - updated



dummy_state = FunctionOptimisationState(players=[None], function=0, ranges=[[0,1]], splits=2)
functions = dummy_state.function_list
join = "/"
logs_path = "logs/Old/FO"
data = pd.read_csv(logs_path + join + "collective_tree_logs.csv")
n_buckets = 128
f_max_locations = [0.5,0.867,None,0.1,0.1]

for f_index in [0,1,2,3,4]:
   agents_names = data["player"].unique()
   print(agents_names)
   c_params = data["c_param"].unique()
   c_params.sort()
   generic_name = "MCTS_c"
   agents_names = [x for x in agents_names]
   names = []
   data_list = []

   #order names
   for c in c_params:
      string_c = str(c)
      if string_c.split(".")[-1] == "0":
            string_c = string_c[:-2]
      names.append(generic_name + string_c)
   for name in agents_names:
      if "SE_MCTS" in name:
            names.append(name)
   for name in names:
      temp_data = exps.get_subset(data, name, f_index)
      data_list.append(temp_data)
   
   treated_names = []
   for it,st in enumerate(names):
      new_string = st.replace("_c"," C = ")
      new_string = new_string.replace("1.414","\u221A\u03052\u0305fred")
      new_string = new_string.split("fred")[0]
      if "SE_MCTS" in new_string:
            if "2600" in new_string:
               new_string = "SIEA_MCTS 2600 iterations"
            else:
               new_string = "SIEA_MCTS 5000 iterations"
      treated_names.append(new_string)
   #plot = exps.show_search(data_list, functions[f_index], "", 3, n_buckets = n_buckets, subplot_titles = treated_names+["Function "+str(f_index+1)], max_x_location=f_max_locations[f_index], y_ref_value=None)
   plot = exps.show_search(data_list, functions[f_index], "Exploration and exploitation distribution for Function " + str(f_index+1) + "." , 3, n_buckets = n_buckets, subplot_titles = ["Function "+str(f_index+1)] + treated_names, max_x_location=f_max_locations[f_index], y_ref_value=None)
   plot.write_image(os.path.join(logs_path, "F" + str(f_index+1) + "_results"+ str(n_buckets) + '.png'))#, width=800, height=800)
   plot.show()

In [ ]:
#plot depth by iteration

dummy_state = FunctionOptimisationState(players=[None], function=0, ranges=[[0,1]], splits=2)
functions = dummy_state.function_list
join = "/"
logs_path = "logs/Old/FO"
output_name = "collective_tree_logs.csv"
data = pd.read_csv(logs_path + join + "collective_tree_logs.csv")
n_buckets = 400
f_max_locations = [0.5,0.867,None,0.1,0.1]

for f_index in [0]:#,1,2,3,4]:
   agents_names = data["player"].unique()
   #print(agents_names)
   c_params = data["c_param"].unique()
   c_params.sort()
   generic_name = "MCTS_c"
   agents_names = [x for x in agents_names]
   names = []
   data_list = []

   #order names
   for c in c_params:
      string_c = str(c)
      if string_c.split(".")[-1] == "0":
         string_c = string_c[:-2]
      names.append(generic_name + string_c)
   for name in agents_names:
      if "SE_MCTS" in name:
         names.append(name)
   for name in names:
      temp_data = exps.get_subset(data, name, f_index)
      data_list.append(temp_data)
   
   treated_names = []
   temp_names = [x for x in names]
   for it,st in enumerate(temp_names):
      new_string = st.replace("_c"," C = ")
      new_string = new_string.replace("1.414","sqrt(2)fred")
      new_string = new_string.split("fred")[0]
      if "SE_MCTS" in new_string:
         if "2600" in new_string:
            new_string = "SE_MCTS partial simulations"
            #continue
         else:
            new_string = "SIEA_MCTS complete simulations"
      treated_names.append(new_string)

   for i,df in enumerate(data_list):
      df['player'] = df['player'].replace([names[i]],treated_names[i])

   for i,name in enumerate(treated_names):
      if name == "SE_MCTS partial simulations":
         del data_list[i]
   
   #plot = exps.show_search(data_list, functions[f_index], "", 3, n_buckets = n_buckets, type="divisions")#, subplot_titles = treated_names+["Function "+str(f_index+1)], max_x_location=f_max_locations[f_index], y_ref_value=None)
   plot = exps.show_search_depth(data_list, "Average expansion depth by iteration. Function " + str(f_index+1), 30, 10)
   #depth_plot.write_image(os.path.join(logs_path, "depth" + str(f_index) + '.png'), width=600, height=350)
   #plot = show_search2(data_list, functions[f_index], "", 3, n_buckets = 200, type="divisions", subplot_titles = agents_names+["Function "+str(f_index)])
   plot.show()
   #depth_plot.write_image(os.path.join(logs_path, "depth_f" + str(f_index) + '.png'), width=800, height=400)
   #plot.write_image(os.path.join(logs_path, "depth_histo_max_2k_f" + str(f_index) + '.png'), width=800, height=800)
   #plot.write_image(os.path.join(logs_path, "Average_depth_by_iteration_f" + str(f_index+1) + '.png'), width=800, height=400)
   colors = ["#000000"
                , "#B10909" #red
                ,  "#5B8C5A"#green
                ,"#56638A" #blue-purple
                , "#EC7316" #orange
                ,  "#FC738C" ] #pink

In [ ]:
def plot_surface(state):
    assert state.function_index in [5,6,7,8,9,10]
    granularity = 1000
    x, y = np.linspace(state.ranges[0][0], state.ranges[0][1], granularity), np.linspace(state.ranges[1][0], state.ranges[1][1], granularity)
    z_dict = {}
    for yi in y:
        z_dict[yi]=[]
        for xi in x:
            z_dict[yi].append(state.function([xi,yi]))
    z = pd.DataFrame(z_dict).values
    fig = go.Figure(data=[go.Surface(z=z, x=x, y=y,
                                    contours = {
                                        #"x": {"show": True, "start": 0, "end": 1, "size": 0.05, "color":"white"},
                                        "z": {"show": True, "start": 0, "end": 1, "size": 0.1, "color":"white"}
                                    })])
    fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                    highlightcolor="limegreen", project_z=True))
    fig.update_layout(#title='Mt Bruno Elevation',
                    autosize=False,
                    scene_camera_eye=dict(x=2, y=1, z=-0.5),
                    width=500, height=500,
                    margin=dict(l=30, r=50, b=10, t=10),
                    scene = {
                        #"xaxis": {"nticks": 5},
                        #"zaxis": {"nticks": 5},
                        "aspectratio": {"x": 1, "y": 1, "z": 1}}
    )
    return fig

def fo_function_analysis_2d(fo_state, max_depth=3, minimum_step_limit = None, print_logs=False):
   """Returns a figure with a 2d histogram plotting the function landscape as MCTS will se it. Manual (Fast)
    Usage example:
    random_player = RandomPlayer()
    dummy_state = FunctionOptimisationState(players=[random_player], function=6, ranges=[[0,1],[0,1]], minimum_step=0.001, splits=3)
    fig = exps.fo_function_analysis_2d(dummy_state, print_logs=True, max_depth=4)
    fig.show()
   """
   #Find evaluation points
   stop = {}
   start = {}
   max_depth_step = {}
   x = {}
   division_size = {}
   dimensions = len(fo_state.ranges)
   if minimum_step_limit is not None:
       minimum_step = minimum_step_limit
   else:
       minimum_step = fo_state.minimum_step

   #splits by dimension
   for d in range(dimensions):
      
      stop[d] = fo_state.ranges[d][1]
      start[d] = fo_state.ranges[d][0]
      max_depth_step[d] = (stop[d]-start[d])/(fo_state.splits**max_depth)
      division_size[d] = stop[d] - start[d]
      while division_size[d] > minimum_step:
         division_size[d] = division_size[d]/fo_state.splits   

      #get central points
      x[d] = []   
      next_start = start[d]
      center_distance = division_size[d]/2
      while next_start+center_distance < stop[d]:
         next_stop = next_start + division_size[d]
         x[d].append(next_start+center_distance) #gets the value in the middle
         next_start = next_stop

   fig_x = {}
   fig_y = {}
   fig_z = {}
   for current_depth in reversed([md+1 for md in range(max_depth)]):
      depth_step = {}
      for d in range(dimensions):
         depth_step[d] = (stop[d]-start[d])/(fo_state.splits**current_depth)

      if max_depth == current_depth:
         granular_x = x[0]
         granular_y = x[1]
      else:
         granular_x = fig_x[current_depth+1]
         granular_y = fig_y[current_depth+1]
         granular_z = {}
         for i in range(len(granular_x)):
            granular_z[(granular_x[i], granular_y[i])] = fig_z[current_depth+1][i]

      all_depth_steps = {}
      for d in range(dimensions):
         all_depth_steps[d] = [[i*depth_step[d], (i+0.5)*depth_step[d], (i+1)*depth_step[d]] for i in range(fo_state.splits**current_depth)]
         all_depth_steps[d][0][0] = start[d]
         all_depth_steps[d][-1][2] = stop[d]

      avg_by_y={}
      for j in granular_y:
         avg_by_y[j] = {}
         steps = 0
         count = 0
         accum = 0
         for i in granular_x:
            if i > all_depth_steps[0][steps][2]:
               avg_by_y[j][all_depth_steps[0][steps][1]] = accum/count
               count = 0
               accum = 0
               steps += 1
            if max_depth == current_depth:
               accum = accum + fo_state.function([i,j])
            else:
               accum = accum + granular_z[(i,j)]
            count += 1
         avg_by_y[j][all_depth_steps[0][steps][1]] = accum/count
         all_x_keys = avg_by_y[j].keys()

      fig_x[current_depth]=[]
      fig_y[current_depth]=[]
      fig_z[current_depth]=[]
      for i in all_x_keys:
         steps = 0
         count = 0
         accum = 0
         for j in avg_by_y.keys():
            if j > all_depth_steps[1][steps][2]:
               fig_x[current_depth].append(i)
               fig_y[current_depth].append(all_depth_steps[1][steps][1])
               fig_z[current_depth].append(accum/count)
               count = 0
               accum = 0
               steps += 1
            accum = accum + avg_by_y[j][i]
            count += 1
         fig_x[current_depth].append(i)
         fig_y[current_depth].append(all_depth_steps[1][steps][1])
         fig_z[current_depth].append(accum/count)

   #create subplots
   plot_pixels = 150
   top_space = 30
   vertical_spacing = 0.03
   n_plots = max_depth
   row_heights = [1/n_plots for _ in range(n_plots)]
   column_widths = [1]
   fig = make_subplots(
      rows=len(row_heights)
      ,cols=len(column_widths)
      ,shared_xaxes=True
      ,vertical_spacing=vertical_spacing
      ,row_heights = row_heights
      ,column_widths = column_widths
      ,subplot_titles=["Tree Depth " + str(d+1) for d in range(max_depth)]
      #,specs=[[{"secondary_y": True}] for _ in range(len(column_widths))]
      )
   
   #add analysis plots
   colorbar_padding = 0.1 #a percetage of the lenght of the colorbars
   show_legend = True
   for d in range(1,max_depth+1):
      if d<3:
         texttemplate = "%{z:.2f}"
      else: texttemplate = ""
      fig.add_trace(
         go.Histogram2d(x=fig_x[d], y=fig_y[d], z=fig_z[d],histfunc ="avg"
            ,name = "Depth " + str(d)
            ,xbins = {"size":(stop[0]-start[0])/fo_state.splits**d}
            ,ybins = {"size":(stop[1]-start[1])/fo_state.splits**d}
            #,color_continuous_scale="gray"
            ,colorscale = [[0, "#000000"], [0.25,"#5B8C5A"], [0.5, "#56638A"],  [0.75, "#EC7316"], [1, "#B10909"]] # [0.5, "#56638A" ],
            ,zmax = 1
            ,zmin = 0
            ,texttemplate = texttemplate
            ,textfont=dict(color="#FFFFFF")
            ,colorbar = dict(
                        tickmode="array",
                        tickvals=[0,0.25,0.5,0.75,1],
                        tick0=0,
                        #dtick=0.1,
                        #nticks=5,
                        #showticklabels=show_legend,
                        #len=(1-colorbar_padding)/max_depth,
                        #y= 1-((1)/max_depth)*(d-1)-(d-1)*vertical_spacing/10,# - (top_space*2/(plot_pixels*n_plots)),
                        #yanchor="top",
                        tickformat = ".2f",
                        #title = str(d)
         )
            )
         ,row=d,col=1)
   show_legend = False
      #fig.add_trace(go.Scatter(x=[start,stop], y=[max(y),max(y)], line=dict(color='royalblue', width=2, dash='dash'),showlegend=False,marker={"color":"blue"}),row=d+1,col=1)

   #update fig layout
   #fig.update_layout(barmode='stack')
   fig.update_layout(margin=dict(l=10, r=10, t=top_space, b=20)
      ,width=plot_pixels+100
      ,height=plot_pixels*n_plots
      ,autosize=False
      ,plot_bgcolor='rgba(0,0,0,0)'
      #,title={"text":"2D Function analysis"}
                ,legend=dict(
                    title = "Formula",
                    orientation="h",
                    yanchor="top",
                    y=-0.65,
                    xanchor="center",
                    x=0.5,  
                    font = dict(family = "Arial", size = 14, color = "black"),
                    bordercolor="LightSteelBlue",
                    borderwidth=2,
                    itemsizing='trace',
                    itemwidth = 30
                    )  
                    )
   #fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='black')
   #fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
   #fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
   # fig.update_xaxes(range=[start,stop])
   fig.update_xaxes(range=[start[0],stop[0]])
   fig.update_yaxes(range=[start[1],stop[1]])
   

   return fig

branching_factor = 2
ranges = [[0,1],[0,1]]
random_player = RandomPlayer()
for f_idx in [5, 6,7,8,9,10]:
    state = FunctionOptimisationState(players=[random_player], function=f_idx, ranges=ranges, splits=branching_factor)

    fig = plot_surface(state)
    fig.show()

    fig = fo_function_analysis_2d(state, max_depth=5,minimum_step_limit=0.001)
    fig.show()

    

